In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shusrith/machine-trainslation")

print("Path to dataset files:", path)

100%|██████████| 185M/185M [00:09<00:00, 21.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shusrith/machine-trainslation/versions/7


In [ ]:
import pandas as pd
from sklearn.utils import shuffle

df = pd.read_csv(f"{path}/EnglishOrSpanish/EnglishOrSpanish/output_joint2.csv")
df = shuffle(df)
df

,English,Spanish
39403,"[2, 270, 607, 1523, 290, 1185, 224, 3, 0, 0, 0...","[2, 302, 1325, 265, 290, 3960, 224, 3, 0, 0, 0..."
30803,"[2, 270, 947, 738, 1185, 224, 3, 0, 0, 0, 0, 0...","[2, 1541, 1345, 18549, 224, 3, 0, 0, 0, 0, 0, ..."
68992,"[2, 395, 716, 429, 272, 842, 290, 804, 224, 3,...","[2, 847, 2972, 3348, 265, 290, 265, 644, 224, ..."
75748,"[2, 361, 334, 947, 2528, 265, 705, 2337, 224, ...","[2, 318, 1541, 410, 2055, 2047, 1580, 224, 3, ..."
75705,"[2, 332, 300, 1396, 300, 2119, 334, 5534, 224,...","[2, 387, 321, 846, 331, 3620, 318, 6324, 224, ..."
...,...,...
63395,"[2, 744, 1124, 404, 1211, 1460, 224, 3, 0, 0, ...","[2, 387, 636, 25855, 690, 355, 1488, 224, 3, 0..."
119128,"[2, 270, 1578, 404, 809, 2579, 674, 4697, 272,...","[2, 333, 4416, 2722, 589, 993, 294, 5291, 265,..."
128260,"[300, 5554, 224, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[224, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
75878,"[2, 1111, 1185, 300, 3744, 272, 290, 224, 3, 0...","[2, 31881, 321, 3253, 265, 290, 224, 3, 0, 0, ..."


In [ ]:
import ast

df["English"] = df["English"].apply(ast.literal_eval)
df["Spanish"] = df["Spanish"].apply(ast.literal_eval)
df

,English,Spanish
39403,"[2, 270, 607, 1523, 290, 1185, 224, 3, 0, 0, 0...","[2, 302, 1325, 265, 290, 3960, 224, 3, 0, 0, 0..."
30803,"[2, 270, 947, 738, 1185, 224, 3, 0, 0, 0, 0, 0...","[2, 1541, 1345, 18549, 224, 3, 0, 0, 0, 0, 0, ..."
68992,"[2, 395, 716, 429, 272, 842, 290, 804, 224, 3,...","[2, 847, 2972, 3348, 265, 290, 265, 644, 224, ..."
75748,"[2, 361, 334, 947, 2528, 265, 705, 2337, 224, ...","[2, 318, 1541, 410, 2055, 2047, 1580, 224, 3, ..."
75705,"[2, 332, 300, 1396, 300, 2119, 334, 5534, 224,...","[2, 387, 321, 846, 331, 3620, 318, 6324, 224, ..."
...,...,...
63395,"[2, 744, 1124, 404, 1211, 1460, 224, 3, 0, 0, ...","[2, 387, 636, 25855, 690, 355, 1488, 224, 3, 0..."
119128,"[2, 270, 1578, 404, 809, 2579, 674, 4697, 272,...","[2, 333, 4416, 2722, 589, 993, 294, 5291, 265,..."
128260,"[300, 5554, 224, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[224, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
75878,"[2, 1111, 1185, 300, 3744, 272, 290, 224, 3, 0...","[2, 31881, 321, 3253, 265, 290, 224, 3, 0, 0, ..."


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

train = TensorDataset(
    torch.tensor(df["English"][:100000].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][:100000].tolist(), dtype=torch.long),
)
test = TensorDataset(
    torch.tensor(df["English"][100000:].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][100000:].tolist(), dtype=torch.long),
)
train_loader = DataLoader(train, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test, batch_size=256, shuffle=False, drop_last=True)

In [ ]:
import json

with open(f"{path}/EnglishOrSpanish/EnglishOrSpanish/vocab.json", "r") as f:
    vocab = json.load(f)

In [ ]:
import torch
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=4, dropout=0.3):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=14453)
        self.lstm = nn.LSTM(
            hidden_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )

        self.layer_norm = nn.LayerNorm(hidden_size)
        self.hidden_norm = nn.LayerNorm(hidden_size)
        self.cell_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(embedded)
        output = self.layer_norm(output)
        hidden = self.hidden_norm(hidden)
        cell = self.cell_norm(cell)
        return output, hidden, cell

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size

        self.attn = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_output):
        hidden = hidden.unsqueeze(1)
        scores = self.attn(encoder_output)
        scores = torch.tanh(scores + hidden)
        attention = self.v(scores).squeeze(2)
        return nn.functional.softmax(attention, dim=1)

In [ ]:
import torch
import torch.nn as nn


class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, attention, num_layers=4, dropout=0.3):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=23722)

        self.lstm = nn.LSTM(
            2 * hidden_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.attention = attention

        self.layer_norm = nn.LayerNorm(hidden_size)
        self.hidden_norm = nn.LayerNorm(hidden_size)
        self.cell_norm = nn.LayerNorm(hidden_size)

        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x, hidden, cell, encoder_output):
        attention_scores = self.attention(hidden[-1], encoder_output)
        context = torch.bmm(attention_scores.unsqueeze(1), encoder_output)
        embedded = self.dropout(self.embedding(x)).unsqueeze(1)
        embedded = torch.cat([embedded, context], dim=2)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        output = self.layer_norm(output)
        hidden = self.hidden_norm(hidden)
        cell = self.cell_norm(cell)
        output = self.fc(output)
        return output, hidden, cell

In [ ]:
class Seq2Seq:
    def __init__(self, encoder, decoder, device, teacher_forcing_ratio):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.vocab_size = self.decoder.vocab_size
        self.teacher_forcing_ratio = teacher_forcing_ratio

    def train(
        self,
        train_loader,
        test_loader,
        enc_optimizer,
        dec_optimizer,
        criterion,
        device,
        encoder_scheduler,
        decoder_scheduler,
        num_epochs,
    ):
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            self.encoder.train()
            self.decoder.train()
            epoch_loss = 0
            epoch_acc = 0
            progress_bar = tqdm(train_loader, desc="Training", leave=False)

            for src, trg in progress_bar:
                src, trg = src.to(device), trg.to(device)

                batch_size, trg_len = trg.shape

                enc_optimizer.zero_grad()
                dec_optimizer.zero_grad()

                encoder_out, hidden, cell = self.encoder(src)
                outputs = torch.zeros(batch_size, trg_len, self.vocab_size).to(
                    self.device
                )
                x = trg[:, 0]
                for t in range(1, trg_len):
                    output, hidden, cell = self.decoder(x, hidden, cell, encoder_out)
                    output = output.squeeze(1)
                    outputs[:, t, :] = output
                    teacher_force = torch.rand(1).item() < self.teacher_forcing_ratio
                    x = trg[:, t] if teacher_force else output.argmax(dim=1)
                loss = criterion(
                    outputs[:, 1:].reshape(-1, self.vocab_size), trg[:, 1:].reshape(-1)
                )
                loss.backward()
                enc_optimizer.step()
                dec_optimizer.step()

                epoch_loss += loss.item()

                preds = outputs.argmax(dim=2)
                correct = (preds == trg).float().sum().item()
                total = trg.numel()
                batch_acc = correct / total
                epoch_acc += batch_acc

                progress_bar.set_postfix(
                    loss=f"{loss.item():.4f}", acc=f"{batch_acc:.4f}"
                )

            train_loss = epoch_loss / len(train_loader)
            train_acc = epoch_acc / len(train_loader)

            # ---------------- VALIDATION ---------------- #
            self.encoder.eval()
            self.decoder.eval()
            val_epoch_loss = 0
            val_epoch_acc = 0
            progress_bar = tqdm(test_loader, desc="Validating", leave=False)

            with torch.no_grad():
                for src, trg in progress_bar:
                    src, trg = src.to(device), trg.to(device)
                    batch_size, trg_len = trg.shape
                    encoder_out, hidden, cell = self.encoder(src)
                    outputs = torch.zeros(batch_size, trg_len, self.vocab_size).to(
                        self.device
                    )
                    x = trg[:, 0]
                    for t in range(1, trg_len):
                        output, hidden, cell = self.decoder(x, hidden, cell, encoder_out)
                        output = output.squeeze(1)
                        outputs[:, t, :] = output
                        x = output.argmax(dim=1)

                    loss = criterion(
                        outputs[:, 1:].reshape(-1, self.vocab_size),
                        trg[:, 1:].reshape(-1),
                    )
                    val_epoch_loss += loss.item()

                    preds = outputs.argmax(dim=2)
                    correct = (preds == trg).float().sum().item()
                    total = trg.numel()
                    batch_acc = correct / total
                    val_epoch_acc += batch_acc

                    progress_bar.set_postfix(
                        loss=f"{loss.item():.4f}", acc=f"{batch_acc:.4f}"
                    )

            val_loss = val_epoch_loss / len(test_loader)
            val_acc = val_epoch_acc / len(test_loader)
            print(
                f"Train loss : {train_loss}, Train accuracy : {train_acc}, Val_loss : {val_loss}, val accuracy : {val_acc}"
            )
            encoder_scheduler.step(val_loss)
            decoder_scheduler.step(val_loss)
        return train_loss, train_acc, val_loss, val_acc

    def predict(self, src, trg):
        self.encoder.eval()
        self.decoder.eval()
        with torch.no_grad():
            src, trg = src.to(self.device), trg.to(self.device)
            batch_size, trg_len = trg.shape
            _, hidden, cell = self.encoder(src)
            outputs = torch.zeros(batch_size, trg_len, self.vocab_size).to(self.device)
            x = trg[:, 0]
            for t in range(1, trg_len):
                output, hidden, cell = self.decoder(x, hidden, cell)
                outputs[:, t, :] = output
                x = output.argmax(dim=1)
            return outputs

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)

hidden_size = 128
encoder = Encoder(vocab_size, hidden_size).to(device)
attention = Attention(hidden_size)
decoder = Decoder(vocab_size, hidden_size, attention).to(device)
seq2seq = Seq2Seq(encoder, decoder, device, 0.5)
criterion = nn.CrossEntropyLoss(ignore_index=23722)
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.0005, weight_decay=1e-5)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.0005, weight_decay=1e-5)
encoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    encoder_optimizer, mode="min", factor=0.9, patience=2
)
decoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    decoder_optimizer, mode="min", factor=0.9, patience=2
)

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

seq2seq.train(
    train_loader,
    test_loader,
    encoder_optimizer,
    decoder_optimizer,
    criterion,
    device,
    encoder_scheduler,
    decoder_scheduler,
    20,
)


Epoch 1/20


Train loss : 3.1869095019805127, Train accuracy : 0.5449864227207981, Val_loss : 2.8776751556227693, val accuracy : 0.5265601954277287

Epoch 2/20


Train loss : 2.5729029178619385, Train accuracy : 0.5912727029914533, Val_loss : 2.735896226579109, val accuracy : 0.5425635293756146

Epoch 3/20


Train loss : 2.4556499927471847, Train accuracy : 0.60544259704416, Val_loss : 2.6392043966107663, val accuracy : 0.5573358376352013

Epoch 4/20


Train loss : 2.3439377680802957, Train accuracy : 0.6133146367521366, Val_loss : 2.5018419333263835, val accuracy : 0.5671994069567353

Epoch 5/20


...

Train loss : 1.4221775650978088, Train accuracy : 0.6885539084757837, Val_loss : 1.841837462070769, val accuracy : 0.6309419555063913

Epoch 18/20


Train loss : 1.3723542723900233, Train accuracy : 0.6941033431267809, Val_loss : 1.8345820618941722, val accuracy : 0.6320673549655852

Epoch 19/20


Train loss : 1.3359076937039693, Train accuracy : 0.6970831107549856, Val_loss : 1.7598238634852181, val accuracy : 0.6377692508603738

Epoch 20/20


Train loss : 1.2909511453066116, Train accuracy : 0.7021673566595438, Val_loss : 1.792966458649762, val accuracy : 0.6380592428711898


(1.2909511453066116, 0.7021673566595438, 1.792966458649762, 0.6380592428711898)

In [ ]:
!nvidia-smi

In [ ]:
torch.cuda.empty_cache()